In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Read data
#kaggle competitions download -c titanic

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Read and clean data**

In [ ]:
def Clean_data(df):
    
    df = df.drop(['Name','Ticket'],axis=1)
    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
    df['Age'] = df['Age'].groupby([df['Pclass'], df['Sex']]).apply(lambda x: x.fillna(x.mean()))
    df = df.drop('Cabin',axis=1)
    df = df.drop(df.index[[61,829]])
    df.isnull().sum()
    df.head()
    df = pd.get_dummies(df, prefix=['Embarked'], columns=['Embarked'], drop_first=True)
    df.corr()['Survived'].sort_values()
    X = df.drop('Survived',axis=1)
    y = df['Survived']
    return X,y


df = pd.read_csv("/kaggle/input/titanic/train.csv")
X_train,y_train = Clean_data(df)

# **Scaling data**

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_X_train = scaler.fit_transform(X_train)


In [ ]:
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train.values), 
                       torch.FloatTensor(y_train))


# **Stack as tensor**

# **Training Classification Neural Network**

In [ ]:
EPOCHS = 50
BATCH_SIZE = 1
LEARNING_RATE = 0.001

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(9, 50)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.fc2(dout)
        h2 = self.prelu(a2)
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y
    
model = Net()
opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()



In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=20, shuffle=True, drop_last=True)
model.train()
for e in range(1, 1500):
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        opt.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
       
        
        loss.backward()
        opt.step()
        
        epoch_loss += loss.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f}')
    

# **Test data loader**

In [ ]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test = test.drop(['Name','Ticket'],axis=1)
test['Sex'] = test['Sex'].map({'male': 1, 'female': 0})
test['Age'] = test['Age'].groupby([test['Pclass'], test['Sex']]).apply(lambda x: x.fillna(x.mean()))
test = test.drop('Cabin',axis=1)
test = pd.get_dummies(test, prefix=['Embarked'], columns=['Embarked'], drop_first=True)
test['Fare'] = test['Fare'].groupby([test['Pclass'], test['Sex']]).apply(lambda x: x.fillna(x.mean()))
#scaled_test_X = scaler.fit_transform(test)
  
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(test.values))
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        y_test_pred = model(X_batch)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list = [int(a) for a in y_pred_list]
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
submission['Survived'] = y_pred_list
submission.to_csv('sub.csv', index=False)
submission
